In [42]:
import math
import lucene
import time
import itertools
import numpy as np
from tqdm import tqdm
from java.io import File
import xml.etree.ElementTree as ET
from collections import defaultdict
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import BytesRefIterator
from org.apache.lucene.index import DirectoryReader, Term
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.analysis.core import WhitespaceAnalyzer
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher, BooleanQuery, BooleanClause, TermQuery, BoostQuery
from org.apache.lucene.search.similarities import BM25Similarity, LMJelinekMercerSimilarity, LMDirichletSimilarity
lucene.initVM()

In [43]:
q_name = 'trec678-robust'

In [44]:
index_path = '../../index/'
topicFilePath = f'../../{q_name}.xml'

directory = FSDirectory.open(File(index_path).toPath())
indexReader = DirectoryReader.open(directory)

In [45]:
def query_topics(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    parsed_topics = {}

    for top in root.findall('top'):
        num = top.find('num').text.strip()
        title = top.find('title').text.strip()
        parsed_topics[num] = title

    return parsed_topics

In [46]:
query_all = query_topics(topicFilePath)

In [47]:
def getDocumentVector(luceneDocid, indexReader):

    N = indexReader.numDocs()                   
    
    docVec = defaultdict(lambda: [0, 0]) 
    D = 0                                 
    
    terms = indexReader.getTermVector(luceneDocid, 'CONTENTS')
    iterator = terms.iterator()
    for term in BytesRefIterator.cast_(iterator):
        t = term.utf8ToString()
        tf = iterator.totalTermFreq()  
        df = indexReader.docFreq(Term('CONTENTS', t))  
        D += tf
        docVec[t][0] = tf
        docVec[t][1] = df
    
    docVec = {key: (value[0] / D) * math.log(N / (value[1] + 1)) for key, value in docVec.items()}
    
    total_weight = sum(docVec.values())
    docVec = {key: value / total_weight for key, value in docVec.items()}

    # print(len(docVec), "D", D)
    return docVec


In [48]:
def search(indexReader, query, similarity, top_rel_doc):
    analyzer = EnglishAnalyzer()
    searcher = IndexSearcher(indexReader)
    searcher.setSimilarity(similarity)
    # query = QueryParser("CONTENTS", analyzer).parse(query)

    scoreDocs = searcher.search(query, top_rel_doc).scoreDocs
    
    docids = [scoreDoc.doc for scoreDoc in scoreDocs]

    set_cont = set()
    set_cont = {term for doc in docids for term in getDocumentVector(doc, indexReader).keys()}

    filtered_tok = set()
    for tok in set_cont:
        if tok.isalpha():
            filtered_tok.add(tok)

    

    # return set_cont, docids
    return filtered_tok, docids

In [49]:
def RM3_term_selection(Query, set_ET, docs, indexReader, alpha, lamb, expanded_query_terms):
    
    totalTF = indexReader.getSumTotalTermFreq("CONTENTS")

    Q = Query.split()
    weight = {}

    cf = {}
    for t in set_ET | set(Q):
        T = Term("CONTENTS", t)
        cf[t] = indexReader.totalTermFreq(T)/totalTF

    docVectors = {}
    
    for d in docs:                    
        docVectors[d] = getDocumentVector(d, indexReader)
            
    for w in set_ET:
        p_wr = 0
        for d in docs:                  
            
            p_wd = (lamb*(docVectors[d].get(w,0)) + (1 - lamb)*cf[w]) 
            # p_wd = docVectors[d].get(w,0)     
        
            p_q = 1
            for q in Q:
                # p_q = p_q*docVectors[d].get(q,0)   
                      
                p_q = p_q*(lamb*(docVectors[d].get(q,0)) + (1 - lamb)*cf[q])   


            p_wr = p_wr + p_wd*p_q
        weight[w] = p_wr


    weight = dict(sorted(weight.items(), key=lambda x:x[1], reverse=True)[:expanded_query_terms])
    
    norm = sum(weight.values())
    if norm == 0:
        pass
    else:
        weight = {w:weight[w]/norm for w in weight}
 
    for w in weight.keys() | set(Q):
        weight[w] = (alpha*weight.get(w,0)) + (1-alpha)*(Q.count(w)/len(Q))
   

    temp_list = sorted(weight.items(), key=lambda x:x[1], reverse=True)
    sorted_weights = dict(temp_list)

    return sorted_weights

In [50]:
def expanded_query_BM25(search, RM3_term_selection, k1, b, alpha, top_rel_doc, expanded_query_terms, lamb):

    analyzer = EnglishAnalyzer()
    similarity = BM25Similarity(k1,b)
    expanded_q = []

    i = 0
    # for q in tqdm(query_all.values(), colour='red', desc='Expanding Queries'):
    for q in query_all.values():
     
        i += 1 
        escaped_q = QueryParser('CONTENTS', analyzer).escape(q)      # a few titles had '/' in them which 
        query = QueryParser('CONTENTS', analyzer).parse(escaped_q)
        
        query_terms = [term.strip()[9:] for term in query.toString().split()]
        parsed_q = ' '.join(query_terms)
#         
        
        # expension_term_set, docids = search(indexReader, parsed_q, similarity, top_rel_doc)
        expension_term_set, docids = search(indexReader, query, similarity, top_rel_doc)
        weights = RM3_term_selection(parsed_q, expension_term_set, docids, indexReader, alpha, lamb, expanded_query_terms)
        # print(weights)
    
        booleanQuery = BooleanQuery.Builder()
        for m, n in weights.items():
            t = Term('CONTENTS', m)
            tq = TermQuery(t)
            boostedTermQuery = BoostQuery(tq, float(n))
            BooleanQuery.setMaxClauseCount(4096)
            booleanQuery.add(boostedTermQuery, BooleanClause.Occur.SHOULD)
        booleanQuery = booleanQuery.build()
       
        expanded_q.append(booleanQuery)   

    return expanded_q

In [51]:
def search_retrived(indexReader, Query, Qid, similarity, out_name):

    searcher = IndexSearcher(indexReader)
    searcher.setSimilarity(similarity)
   
    scoreDocs = searcher.search(Query, 1000).scoreDocs             #retrieving top 1000 relDoc
    i = 1
    res = ''

    for scoreDoc in scoreDocs:
        doc = searcher.doc(scoreDoc.doc)
        r = str(Qid) + '\t' + 'Q0' + '\t' + str(doc.get('ID')) + '\t' + str(i) + '\t' + str(scoreDoc.score) + '\t' + str(out_name) + '\n'
        res += r
        i = i+1   

    return res

In [52]:
def run_RM3(top_PRD, expanded_query_terms, alpha, lamb):
    expand_q = expanded_query_BM25(search, RM3_term_selection, k1, b, alpha, top_PRD, expanded_query_terms, lamb)
                                       
    name = 'prm_'
    sim = BM25Similarity(k1,b)
    name = name + 'BM25_' + str(k1) + '_'+ str(b)

    file_name = f'./res_RM3/{q_name}/{q_name}_lambda_' + str(lamb) +'_docs_' + str(top_PRD) + '_terms_' + str(expanded_query_terms) + '_alpha_' + str(alpha) + '.txt'
    out_file = open(file_name, "w")

    res = ''
    for i in tqdm(range(len(query_all)),colour='cyan', desc = 'Re-retrival'):
    # for i in range(len(query_all)):
    
        result =  search_retrived(indexReader, expand_q[i], list(query_all.keys())[i], sim, name)
        res = res + result

    out_file.write(res)
    out_file.close()
    # print("Retrieval Completed - result dumped in", file_name)

In [53]:
k1 = 0.8
b = 0.4

top_PRD = [25]
expanded_query_terms = [50]


alpha = [0.7]
lamb = [1, 0.7]

parameters = list(itertools.product(top_PRD, expanded_query_terms, alpha, lamb))

for num_doc, num_q, alpha, lamb in tqdm(parameters, colour='red'):
    run_RM3(num_doc, num_q, alpha, lamb)

100%|██████████| 2/2 [08:07<00:00, 243.89s/it]
